In [16]:
import warnings
warnings.filterwarnings('ignore')
# from datetime import datetime, date
import numpy as np
import pandas as pd
# import yfinance as yf

In [17]:
# A empresa e Indicadores fundamentalistas dados do statusinvest.com.br
#(pode ter atraso, dados não são pegos diretamente no SI)

url = 'https://raw.githubusercontent.com/BDonadelli/Codigos-em-financas/main/data/SI_Acoes.csv'
pd.set_option('display.max_columns', None)
fundsSI = pd.read_csv(url,sep=';' , decimal=',' ,thousands ='.' )

In [18]:
fundsSI.columns

Index(['TICKER', 'PRECO', 'DY', 'P/L', 'P/VP', 'P/ATIVOS', 'MARGEM BRUTA',
       'MARGEM EBIT', 'MARG. LIQUIDA', 'P/EBIT', 'EV/EBIT',
       'DIVIDA LIQUIDA / EBIT', 'DIV. LIQ. / PATRI.', 'PSR', 'P/CAP. GIRO',
       'P. AT CIR. LIQ.', 'LIQ. CORRENTE', 'ROE', 'ROA', 'ROIC',
       'PATRIMONIO / ATIVOS', 'PASSIVOS / ATIVOS', 'GIRO ATIVOS',
       'CAGR RECEITAS 5 ANOS', 'CAGR LUCROS 5 ANOS', ' LIQUIDEZ MEDIA DIARIA',
       ' VPA', ' LPA', ' PEG Ratio', ' VALOR DE MERCADO'],
      dtype='object')

In [19]:
fundsSI =  fundsSI[fundsSI[' LIQUIDEZ MEDIA DIARIA'] > 300000] 
fundsSI =  fundsSI[(fundsSI[' LPA'] > 0) & (fundsSI[' VPA'] > 0) ]
# fundsSI

In [20]:
fundsSI['valor_intrinseco'] = np.sqrt(22.5 * fundsSI[' LPA'] * fundsSI[' VPA'])
fundsSI['Delta'] = fundsSI['valor_intrinseco'] / fundsSI['PRECO'] -1
fundsSI["Rank"]   = fundsSI['Delta'].rank(ascending=True, method="min")
fundsSI.sort_values(by="Rank", ascending=False, inplace=True)
fundsSI.reset_index(inplace=True)
fundsSI.index = fundsSI.index + 1

top20 = fundsSI[['TICKER','PRECO' , 'valor_intrinseco' , 'Rank' , 'Delta' , ' LPA' , ' VPA' ]].head(20)
top20

,TICKER,PRECO,valor_intrinseco,Rank,Delta,LPA,VPA
1,LUPA3,2.17,8.714987,192.0,3.016123,0.87,3.88
2,GOAU4,10.36,32.252023,191.0,2.113130,2.66,17.38
3,GOAU3,10.50,32.252023,190.0,2.071621,2.66,17.38
4,TECN3,3.21,9.446904,189.0,1.942961,0.74,5.36
5,MEAL3,2.16,5.982725,188.0,1.769780,0.41,3.88
6,HBOR3,3.04,8.386596,187.0,1.758749,0.30,10.42
7,GGBR3,20.35,54.774447,186.0,1.691619,4.63,28.80
8,HBRE3,5.58,14.570089,185.0,1.611127,0.51,18.50
9,BRAP3,20.01,49.797741,184.0,1.488643,5.40,20.41
10,GGBR4,22.36,54.774447,183.0,1.449662,4.63,28.80


In [21]:
fundsSI =  fundsSI[(fundsSI['P/L'] > 0)]
top20 = fundsSI[['TICKER','PRECO' , 'valor_intrinseco' , 'Rank' , 'Delta' , ' LPA' , ' VPA' ]].head(20)
top20

,TICKER,PRECO,valor_intrinseco,Rank,Delta,LPA,VPA
1,LUPA3,2.17,8.714987,192.0,3.016123,0.87,3.88
2,GOAU4,10.36,32.252023,191.0,2.113130,2.66,17.38
3,GOAU3,10.50,32.252023,190.0,2.071621,2.66,17.38
4,TECN3,3.21,9.446904,189.0,1.942961,0.74,5.36
5,MEAL3,2.16,5.982725,188.0,1.769780,0.41,3.88
6,HBOR3,3.04,8.386596,187.0,1.758749,0.30,10.42
7,GGBR3,20.35,54.774447,186.0,1.691619,4.63,28.80
8,HBRE3,5.58,14.570089,185.0,1.611127,0.51,18.50
9,BRAP3,20.01,49.797741,184.0,1.488643,5.40,20.41
10,GGBR4,22.36,54.774447,183.0,1.449662,4.63,28.80


In [22]:
fundsSI =  fundsSI[(fundsSI['CAGR RECEITAS 5 ANOS'] > 0) ]
top20 = fundsSI[['TICKER','PRECO' , 'valor_intrinseco' , 'Rank' , 'Delta' , ' LPA' , ' VPA' ]].head(20)
top20

,TICKER,PRECO,valor_intrinseco,Rank,Delta,LPA,VPA
2,GOAU4,10.36,32.252023,191.0,2.113130,2.66,17.38
3,GOAU3,10.50,32.252023,190.0,2.071621,2.66,17.38
4,TECN3,3.21,9.446904,189.0,1.942961,0.74,5.36
5,MEAL3,2.16,5.982725,188.0,1.769780,0.41,3.88
6,HBOR3,3.04,8.386596,187.0,1.758749,0.30,10.42
7,GGBR3,20.35,54.774447,186.0,1.691619,4.63,28.80
8,HBRE3,5.58,14.570089,185.0,1.611127,0.51,18.50
10,GGBR4,22.36,54.774447,183.0,1.449662,4.63,28.80
12,NTCO3,16.54,39.493123,181.0,1.387734,3.43,20.21
13,SAPR3,5.01,11.827383,180.0,1.360755,0.99,6.28


In [24]:
fundsSI =  fundsSI[(fundsSI['CAGR LUCROS 5 ANOS'] > 0) ]
top20 = fundsSI[['TICKER','PRECO' , 'valor_intrinseco' , 'Rank' , 'Delta' , ' LPA' , ' VPA' ]].head(20)
top20

,TICKER,PRECO,valor_intrinseco,Rank,Delta,LPA,VPA
5,MEAL3,2.16,5.982725,188.0,1.769780,0.41,3.88
12,NTCO3,16.54,39.493123,181.0,1.387734,3.43,20.21
13,SAPR3,5.01,11.827383,180.0,1.360755,0.99,6.28
16,ALOS3,24.52,56.101845,177.0,1.288003,5.46,25.62
17,NEOE3,21.16,48.354565,176.0,1.285187,4.28,24.28
18,CMIG4,12.07,27.554310,175.0,1.282876,3.04,11.10
20,SAPR11,26.19,59.036120,173.0,1.254147,4.93,31.42
21,EUCA4,17.70,39.888636,172.0,1.253595,2.97,23.81
22,SAPR4,5.26,11.827383,171.0,1.248552,0.99,6.28
25,COCE5,42.23,92.970298,168.0,1.201523,7.05,54.49


In [25]:
fundsSI =  fundsSI[(fundsSI['DIVIDA LIQUIDA / EBIT'] <4) ]
top20 = fundsSI[['TICKER','PRECO' , 'valor_intrinseco' , 'Rank' , 'Delta' , ' LPA' , ' VPA' ]].head(20)
top20

,TICKER,PRECO,valor_intrinseco,Rank,Delta,LPA,VPA
5,MEAL3,2.16,5.982725,188.0,1.769780,0.41,3.88
12,NTCO3,16.54,39.493123,181.0,1.387734,3.43,20.21
13,SAPR3,5.01,11.827383,180.0,1.360755,0.99,6.28
16,ALOS3,24.52,56.101845,177.0,1.288003,5.46,25.62
17,NEOE3,21.16,48.354565,176.0,1.285187,4.28,24.28
18,CMIG4,12.07,27.554310,175.0,1.282876,3.04,11.10
20,SAPR11,26.19,59.036120,173.0,1.254147,4.93,31.42
21,EUCA4,17.70,39.888636,172.0,1.253595,2.97,23.81
22,SAPR4,5.26,11.827383,171.0,1.248552,0.99,6.28
26,ROMI3,12.56,27.447787,167.0,1.185333,2.62,12.78


In [23]:
# linhas_com_nan = top20[top20.isna().any(axis=1)]
# linhas_com_nan

### Porque 22,5?

O número 22,5 na Fórmula de Graham é um fator de ponderação que tem um propósito específico. Essa constante foi escolhida por Graham pra ajustar a avaliação do preço justo de uma ação com base na taxa de crescimento anual esperada da empresa.

O número 22,5 é, na verdade, o resultado da multiplicação de 8,5 por 2,65 (8,5 x 2,65 = 22,5). O número 8,5 é a base que Graham considerou razoável pra uma empresa com taxa de crescimento zero, ou seja, uma empresa que não cresce.

Já o número 2,65 representa a média do retorno exigido pelos investidores no mercado de ações durante a época de Graham, que era de aproximadamente 4,4% acima da taxa de retorno dos títulos do Tesouro dos Estados Unidos.

O fator 22,5 ajuda a ajustar o preço justo com base no crescimento da empresa e na expectativa de retorno dos investidores.

Esse ajuste garante que a Fórmula de Graham considere a taxa de crescimento anual esperada e reflita uma avaliação mais realista do preço justo de uma ação.

No próximo tópico, abordaremos a calculadora do preço justo usando a Fórmula de Graham.